<a href="https://colab.research.google.com/github/henryouly/ESP8266-CAR/blob/master/experimental/colab/story_teller_v2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [61]:
# @title Install dependencies

!pip install -qU langgraph langchain langchain_openai langchain_anthropic langchainhub python-dotenv

In [62]:
# @title Imports

import os
import openai
import numpy as np
import pandas as pd
import dotenv

from langchain import hub, PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI, OpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.openai_functions import create_structured_output_runnable, create_openai_fn_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_openai_functions_agent
from langgraph.prebuilt import create_agent_executor

from pydantic import BaseModel, Field
from typing import Type
from bs4 import BeautifulSoup
import requests
import json
from langchain.schema import SystemMessage

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict, Optional
import operator


In [63]:
# @title Read API keys from colab secrets

from google.colab import userdata
for key in ['LANGCHAIN_API_KEY', 'ANTHROPIC_API_KEY', 'OPENAI_API_KEY']:
  os.environ[key] = userdata.get(key)


In [64]:
# @title Setup LangSmith
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'story_teller'

# Config

In [43]:
MAX_TITLE_LENGTH = 50 # @param {"type": "integer"}
MAX_SCRIPT_LENGTH = 2000 # @param {"type": "integer"}
MAX_ABSTRACT_LENGTH = 400 # @param {"type": "integer"}
MAX_CHARACTERS = 5 # @param {"type": "integer"}

In [44]:
# @title idea_to_script_prompt (unmodified)

INSTRUCTION_IDEA_TO_SCRIPT = '''
You're a professional writer. Your task is to create a narrative text based on
an idea, outline, or reference story, news, fact, data, etc. The narrative text
can be used to tell a short story, produce a comic strip, a short film, or a
slideshow.

Please think step-by-step.

Step 1: Please identify the type of the content: story or non-story.

Step 2: Use your greatest creativity to write a narrative text no longer than
{max_script_length} Unicode characters, in {output_language}. Please use a
"{tone}" writing style.

For story content, you need to carefully design characters with distinctive
personalities (can be human, animals, anthropomorphic characters, etc.), and
design the most attractive plot (refer to the common screenwriting techniques
such as suspense, twists, reversals, reverse order, coincidence, etc.).

For non-story content, usually you don't have to consider the characters, but
rather how to explain the product, knowledge, process, information, reasoning,
etc. more clearly, accurately, completely and persuasively.

Step 3: Make sure your text is not too simple, not boring, not empty or not too
short. Do NOT format the text, do NOT make lines or paragraphs, do NOT include
any special characters. Title is optional.

Step 4: make sure your narrative text is in {output_language}. If not, translate
it into {output_language}.

'''

idea_to_script_prompt = ChatPromptTemplate.from_messages([("system", INSTRUCTION_IDEA_TO_SCRIPT), ("user", """{user_input}""")])

In [45]:
# @title idea_to_script_prompt_v2

system = (
    """
    You're a professional writer. Your task is to create a narrative text based on
    an idea, outline, or reference story, news, fact, data, etc. The narrative text
    can be used to tell a short story, produce a comic strip, a short film, or a
    slideshow.

    Please follow the below rules to create the narrative.

    1/ Please choose one type of the content: story or non-story.

    2/ Use your greatest creativity to write a narrative text no longer than
    {max_script_length} Unicode characters.

    3/ Use a "{tone}" writing style.

    4/ For story content, you need to carefully design characters with distinctive
    personalities (can be human, animals, anthropomorphic characters, etc.), and
    design the most attractive plot (refer to the common screenwriting techniques
    such as suspense, twists, reversals, reverse order, coincidence, etc.).

    5/ For non-story content, usually you don't have to consider the characters, but
    rather how to explain the product, knowledge, process, information, reasoning,
    etc. more clearly, accurately, completely and persuasively.

    6/ Text is not too simple, not boring, not empty or not too
    short. Do NOT format the text, do NOT make lines or paragraphs, do NOT include
    any special characters. Title is optional.

    7/ Make sure your narrative text is in {output_language}. If not, translate
    it into {output_language}.
    """
)

idea_to_script_prompt_v2 = ChatPromptTemplate.from_messages([("system", system), ("user", """{user_input}""")])

In [46]:
# @title script_to_summary (unmodified)

INSTRUCTION_SCRIPT_TO_SUMMARY = '''
You're a professional comic book writer. Before creating a comic strip, your
first task is to distill the title, the abstract, and the characters of a short
story from an existing narrative full text.

Regardless of the language of the original text, the title and the abstract must
be in {output_language}. And, the language of character's name and description
must be in English.

Please think step-by-step.

Step 1: summarize the story and creates a title and an abstract. The title
should not exceed {max_title_length} characters in length. The abstract should
not exceed {max_abstract_length} characters in length.

Step 2: make sure the title and the abstract are in {output_language}. If not,
translate them into {output_language}.

Step 3: list single human or humanoid character that appears in the story, give
each character a unique English name. You should list no more than
{max_characters} characters. If there are too many individuals in the story,
keep the most important {max_characters}. If the story tells a fact, a piece of
knowledge, or a piece of information and does not mention any human or humanoid
characters, then use an empty character list.

Good character examples include: "Alice", "Bob", "Albert Einstein", "Mr. Apple",
"Dr. Who", etc.

Do not list a group as a character. "family", "parents", "group of warriors",
"the audience", etc. are not suitable character names.

Try to avoid non-humanoid characters. "stone", "car", "the earth", "machine
learning", etc. are not suitable character names unless they are personified in
the story.

Step 4: write an accurate, detailed appearance description in English for each
character (if there is any), following this template: male/female/unknown
gender, era, ethnicity/unknown ethnicity, age, body type, eye size, skin color,
hair color, clothing type, clothing color, accessories, other visible features.

Good examples of character description include:

"A humanoid elephant, male, 18th century, unknown race, unknown age, tall,
strong, small eyes, gray skin, no hair, wearing a white sweatshirt and a blue
baseball cap."

"Little girl, modern, African American, age 14, short, slightly chubby, big
eyes, dark skin, dark hair, wearing a red jacket, blue jeans, and a plaid
scarf."
'''

script_to_summary_prompt = ChatPromptTemplate.from_messages([("system", INSTRUCTION_SCRIPT_TO_SUMMARY), ("user", """{script}""")])

In [47]:
df = pd.read_csv('/content/story_teller_test_cases.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/story_teller_test_cases.csv'

In [ ]:
# Models

from langchain_anthropic import ChatAnthropic

gpt_llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)
claude_llm = ChatAnthropic(model='claude-3-haiku-20240307')

# ChatGPT

In [ ]:
# @title Idea to script using OpenAI function

class Script(BaseModel):
    """A narrative text of a short story."""

    title: Optional[str] = Field(None, description=f"The title of the story, no more than {MAX_TITLE_LENGTH} Unicode characters in length.")
    body: str = Field(..., description=f"The narrative text of the story, no more than {MAX_SCRIPT_LENGTH} characters in length.")
    story_type: str = Field(..., description=f"The story type, can be 'story' or 'non-story")
    personality: Optional[str] = Field(None, description=f"If the type is a story, the designed character's distinctive personality, such as 'human', 'animal', 'anthropomorphic character'")
    plot: Optional[str] = Field(None, description=f"If the type is a story, the most attractive plot in screenwriting techniques, such as 'suspense', 'twists', 'reversals', 'reverse order', 'coincidence'")

script_writer_runnable = create_openai_fn_runnable(
    [Script],
    gpt_llm,
    idea_to_script_prompt,
)

# Extract the body field from the Script object
script_writer = script_writer_runnable | (lambda x: x.body)


In [ ]:
script = script_writer.invoke(
    {"tone": "neutral", "max_script_length": MAX_SCRIPT_LENGTH, "output_language": "English", "user_input": "Can you design an attactive short story about Mr. Apple and Mrs. Banana?"}
)

script

"Mr. Apple and Mrs. Banana were neighbors in a small fruit village. They had different personalities - Mr. Apple was sweet and caring, while Mrs. Banana was sassy and independent. Despite their differences, they always found themselves in amusing situations. One day, Mr. Apple decided to organize a surprise party for Mrs. Banana's birthday. He invited all their fruit friends, including Mr. Strawberry, Ms. Pineapple, and Dr. Watermelon. The party was full of laughter and joy, with everyone bringing their favorite fruit dishes. As the night went on, Mr. Apple presented Mrs. Banana with a beautifully wrapped gift. She opened it eagerly to find a pair of sunglasses, which she had always wanted. Mrs. Banana was moved by his thoughtfulness and realized how much Mr. Apple cared for her. From that day on, their friendship blossomed into something more. They became a happy couple, cherishing their love for each other and embracing their differences."

In [ ]:
results = []

def process_results():
  for i, row in df.iterrows():
    print(f"Processing {row.test_id}")
    results.append(script_writer.invoke({
        "user_input": row.user_input,
        "output_language": row.output_language,
        "tone": row.tone,
        "max_script_length": MAX_SCRIPT_LENGTH
    }))
  result_df = pd.DataFrame([result.__dict__ for result in results])
  result_df.to_csv('story_teller_results.csv')

# process_results()

In [ ]:
# @title script to summary

class Character(BaseModel):
  """A set of story character definitions"""

  name: str = Field(..., description="The name of the story character.")
  description: str = Field(..., description="An accurate, detailed appearance description of the story character.")

class Summary(BaseModel):
  """Save the title, the abstract and the character definitions of the story."""

  title: str = Field(None, description=f"The title of the story, no more than {MAX_TITLE_LENGTH} Unicode characters in length.")
  abstract: str = Field(..., description=f"The abstract of the story, no more than {MAX_ABSTRACT_LENGTH} Unicode characters in length.")
  characters: list[Character] = Field(..., description=f"A set of story character definitions, no more than {MAX_CHARACTERS} story characters.")

generate_summary = create_openai_fn_runnable(
    [Summary],
    llm,
    script_to_summary_prompt,
)

summary = generate_summary.invoke(
    {"max_title_length": MAX_TITLE_LENGTH, "max_abstract_length": MAX_ABSTRACT_LENGTH, "max_characters": MAX_CHARACTERS, "output_language": "English", "script": script.body}
)

summary


Summary(title=None, abstract='Mr. Apple and Mrs. Banana, two fruits with contrasting personalities, form a deep friendship in a fruit market. Together, they inspire unity and resilience in the face of adversity, spreading joy and love wherever they go.', characters=[Character(name='Mr. Apple', description='Male, unknown era, unknown ethnicity, cheerful and optimistic, always has a smile on his face, loves making people laugh'), Character(name='Mrs. Banana', description='Female, unknown era, unknown ethnicity, shy and reserved, loves spending time alone, reading books, and dreaming about adventures')])

# Claude 3 (Kaiku)

In [ ]:
# @title idea_to_script_prompt v2

system = ("""
    You're a professional writer. Your task is to create a narrative text based on
    an idea, outline, or reference story, news, fact, data, etc. The narrative text
    can be used to tell a short story, produce a comic strip, a short film, or a
    slideshow.

    Please follow the below rules to create the narrative.

    1/ Please choose one type of the content: story or non-story.

    2/ Use your greatest creativity to write a narrative text no longer than
    {max_script_length} Unicode characters.

    3/ Use a "{tone}" writing style.

    4/ The language of the narrative text must be in {output_language}.

    5/ For story content, you need to carefully design characters with distinctive
    personalities (can be human, animals, anthropomorphic characters, etc.), and
    design the most attractive plot (refer to the common screenwriting techniques
    such as suspense, twists, reversals, reverse order, coincidence, etc.).

    6/ For non-story content, usually you don't have to consider the characters, but
    rather how to explain the product, knowledge, process, information, reasoning,
    etc. more clearly, accurately, completely and persuasively.

    7/ Text is not too simple, not boring, not empty or not too short.

    8/ Do NOT format the text, do NOT make lines or paragraphs, do NOT include
    any special characters.

    9/ Skip the preamble and provide only the narrative text.

    10/ Finally, call the function save_script with the final text. Call the function once and only once.
    """
)

tools = ("""
    In this environment you have access to a set of tools you can use to answer the user's question.

    You may call them like this:
    <function_calls>
    <invoke>
    <tool_name>$TOOL_NAME</tool_name>
    <parameters>
    <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
    ...
    </parameters>
    </invoke>
    </function_calls>

    Here are the tools available:
    <tools>
    <tool_description>
    <tool_name>save_script</tool_name>
    <description>Saves a narrative text of a short story.</description>
    <parameters>
    <parameter>
    <name>title</name>
    <type>string</type>
    <description>The title of the story.</description>
    </parameter>
    <parameter>
    <name>body</name>
    <type>string</type>
    <description>The narrative text of the story.</description>
    </parameter>
    </parameters>
    </tool_description>
    """
)

idea_to_script_prompt_claude = ChatPromptTemplate.from_messages([("system", system + tools), ("user", """{user_input}""")])

In [ ]:
# @title idea_to_script_prompt v2
#
# This new prompt looks working better for
# 1. Enforcing the output language
# 2. Encourage the function call at the end.

system = ("""
    You're a professional writer. Your task is to create a narrative text based on
    an idea, outline, or reference story, news, fact, data, etc. The narrative text
    can be used to tell a short story, produce a comic strip, a short film, or a
    slideshow.

    Please follow the below rules to create the narrative.

    2/ Use your greatest creativity to write a narrative text no longer than
    {max_script_length} Unicode characters.

    3/ The writing style and tone of the narrative must be "{tone}".

    4/ Your output must be in {output_language}.

    5/ Text is not too simple, not boring, not empty or not too short.

    Please follow the below steps to create the narrative.

    Firstly, choose one type of the content: story or non-story.

    For story content, you need to carefully design characters with distinctive
    personalities (can be human, animals, anthropomorphic characters, etc.), and
    design the most attractive plot (refer to the common screenwriting techniques
    such as suspense, twists, reversals, reverse order, coincidence, etc.).

    For non-story content, usually you don't have to consider the characters, but
    rather how to explain the product, knowledge, process, information, reasoning,
    etc. more clearly, accurately, completely and persuasively.

    Finally, call the function save_script with the final text. Call the function once and only once.
    """
)

tools = ("""
    In this environment you have access to a set of tools you can use to answer the user's question.

    You may call them like this:
    <function_calls>
    <invoke>
    <tool_name>$TOOL_NAME</tool_name>
    <parameters>
    <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
    ...
    </parameters>
    </invoke>
    </function_calls>

    Here are the tools available:
    <tools>
    <tool_description>
    <tool_name>save_script</tool_name>
    <description>Saves a narrative text of a short story.</description>
    <parameters>
    <parameter>
    <name>title</name>
    <type>string</type>
    <description>The title of the story.</description>
    </parameter>
    <parameter>
    <name>body</name>
    <type>string</type>
    <description>The narrative text of the story.</description>
    </parameter>
    </parameters>
    </tool_description>
    """
)

idea_to_script_prompt_claude = ChatPromptTemplate.from_messages([("system", system + tools), ("user", """{user_input}""")])

In [48]:
# @title idea_to_script_prompt v3
#
# This new prompt looks working better for
# 1. Use XML tags to specify input and gauge the output.
# 2. Improvement to the language
#
# Reference:
# https://docs.anthropic.com/claude/docs/use-xml-tags
# https://docs.anthropic.com/claude/docs/let-claude-think

system = ("""
    You're a professional writer. Your task is to create a narrative text based on
    an idea, outline, or reference story, news, fact, data, etc. The narrative text
    can be used to tell a short story, produce a comic strip, a short film, or a
    slideshow.

    Please follow the below rules to create the narrative.

    1. The narrative text must be no longer than {max_script_length} Unicode characters.
    2. The writing style and tone of the narrative must be "{tone}".
    3. Your output must be in {output_language}.
    4. Be creative.
    5. Text is not too simple, not boring, not empty or not too short.

    To create the narrative, use the below procedure below.

    Firstly, choose one type of the content: story or non-story.

    For story content, you need to carefully design characters with distinctive
    personalities (can be human, animals, anthropomorphic characters, etc.), and
    design the most attractive plot (refer to the common screenwriting techniques
    such as suspense, twists, reversals, reverse order, coincidence, etc.).

    For non-story content, usually you don't have to consider the characters, but
    rather how to explain the product, knowledge, process, information, reasoning,
    etc. more clearly, accurately, completely and persuasively.

    Use the below output format:

    <narrative>
    <title>
    {{TITLE OF THE NARRATIVE}}
    </title>

    <body>
    {{BODY OF THE NARRATIVE}}
    </body>
    </narrative>
    """
)

idea_to_script_prompt_claude = ChatPromptTemplate.from_messages([("system", system), ("user", """{user_input}""")])

In [49]:
import re

def extract_between_tags(tag, string, strip=False):
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    return [e.strip() for e in ext_list] if strip else ext_list


In [50]:
script_writer_claude = idea_to_script_prompt_claude | claude_llm | (lambda x: extract_between_tags("body", x.content, True)[0])

In [51]:
script_text = script_writer_claude.invoke(
    {
        "tone": "neutral",
        "max_script_length": MAX_SCRIPT_LENGTH,
        "output_language": "English",
        "user_input": "Can you design an attactive short story about Mr. Apple and Mrs. Banana?"
    }
)

In [53]:
script_text = script_writer_claude.invoke(
    {
        "tone": "Children's",
        "max_script_length": MAX_SCRIPT_LENGTH,
        "output_language": "English",
        "user_input": "请写一个专⻔帮助英语不好的孩子学英语的小故事,这个小故事里使用的词汇特别简单,并且,这个小故事的文本很好地展示了英文单词 take 在不同情境下的不同用法。"
    }
)

In [54]:
from IPython.display import Markdown

Markdown(script_text)

Tom was a young boy who loved to learn new things. But he had a hard time learning English. The words were so strange and hard to remember!

One day, Tom's mom said, "Tom, it's time to take a break from your English homework. Let's go outside and take a walk."

Tom was happy to take a break. He took his mom's hand and they went outside. As they walked, Tom saw a bird take flight and fly high in the sky.

"Wow, look at that bird take off!" said Tom. "I wish I could take English lessons and learn to speak it well like that bird takes to the sky."

His mom smiled and said, "Don't worry, Tom. With practice, you'll take to English just like that bird. It just takes time and effort."

When they got home, Tom sat down to take another look at his English workbook. This time, the words didn't seem so scary. He took his pencil and started practicing writing the new words.

From that day on, Tom took English more seriously. He took his time and practiced every day. Slowly but surely, he started to take in the new words and phrases.

One day, his teacher said, "Tom, you've really taken to English! I'm so proud of the progress you've made."

Tom smiled, happy that he had taken the time to learn. He knew that with hard work, he could take on any challenge, even learning a new language.

# Eval using langsmith API without scoring

In [26]:
os.environ['LANGCHAIN_PROJECT'] = 'story_teller_eval'

In [59]:
# @title Eval Config

EVAL_MODEL = "claude-3-haiku-20240307" # @param ["gpt-3.5-turbo-16k", "claude-3-haiku-20240307"] {type:"string"}
EVAL_DATASET = "story_teller_test_v1_0314" # @param ["script_writer_single_test_joke", "story_teller_test_v1_0314"]

_eval_model_chain = script_writer if EVAL_MODEL == "gpt-3.5-turbo-16k" else script_writer_claude
_eval_concurrency_level = 5 if EVAL_MODEL == "gpt-3.5-turbo-16k" else 3

In [60]:
import langsmith
from langchain import chat_models, prompts
from langchain.schema import output_parser

import datetime
uid = datetime.datetime.now().strftime('%Y%m%d%H%M%S')


client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name=EVAL_DATASET,
    llm_or_chain_factory=_eval_model_chain,
    project_name=f"{EVAL_MODEL}_{uid}",
    concurrency_level=_eval_concurrency_level,
    verbose=True,
)

View the evaluation results for project 'claude-3-haiku-20240307_20240320030048' at:
https://smith.langchain.com/o/7ba29406-b38b-5407-bd7b-dbefcd2e2fa3/datasets/30870756-6b25-4aa2-9ae3-1ccacfb1f263/compare?selectedSessions=400f8aa4-40e6-4021-b632-b771479b6d05

View all tests for Dataset story_teller_test_v1_0314 at:
https://smith.langchain.com/o/7ba29406-b38b-5407-bd7b-dbefcd2e2fa3/datasets/30870756-6b25-4aa2-9ae3-1ccacfb1f263
[------------------------------------------------->] 13/13
 Experiment Results:
       error  execution_time                                run_id
count      0           13.00                                    13
unique     0             NaN                                    13
top      NaN             NaN  18b45e91-5eb9-48a9-993f-18bfd6c41a2a
freq     NaN             NaN                                     1
mean     NaN           13.82                                   NaN
std      NaN           13.30                                   NaN
min      NaN        